In [ ]:
%reset

### !!!! Run Combine_features.ipynb before running this file !!!. 

### Then execute cell by cell for part 2:

In [ ]:
import cv2
import imutils
import matplotlib
import numpy as np
import pandas as pd
from functools import reduce

In [ ]:
features_nonfaces=np.load('features_nonfaces.npy')
features_faces=np.load('features_faces.npy')

In [ ]:
def decision_stump(feature_val,y,m,weights,false_neg_penalty=0.5,false_pos_penalty=0.5,err_atol=1e-08,err_rtol=1e-02):
    feature_val_inds = feature_val.argsort()
    feature_val = feature_val[feature_val_inds[::1]]
    y_sorted = y[feature_val_inds[::1]]
    my_array=np.unique(feature_val)
    Tau=(my_array+np.roll(my_array,shift=-1))/2
    Tau=np.concatenate(([my_array[0]-1],Tau))
    Tau[-1]=my_array[-1]+1
    Margin = np.concatenate(([1],np.roll(my_array,shift=-1)-my_array))
    Margin[-1]=1
    #weights=weights*(false_neg_penalty+false_pos_penalty)*2
    error_samples=np.bitwise_xor(np.greater_equal(feature_val,Tau[:,np.newaxis])\
                           ,(y_sorted>0)[np.newaxis,:])
    weighted_err=np.multiply.reduce([error_samples, weights[np.newaxis,:],\
                                     np.where(y_sorted==1,false_neg_penalty,false_pos_penalty)[np.newaxis,:]],0).sum(axis=1)
    Toggle=np.greater(weighted_err,0.5)

    if np.any(Toggle):
        error_samples=np.bitwise_xor(error_samples,Toggle[:,np.newaxis]).astype('bool')
        weighted_err=np.multiply.reduce([error_samples, weights[np.newaxis,:],\
                                     np.where(y_sorted==1,false_neg_penalty,false_pos_penalty)[np.newaxis,:]],0).sum(axis=1)


    #min_err_index = np.where(np.isclose(weighted_err,weighted_err[np.argmax(np.absolute(weighted_err-0.5))]\
    #                                ,rtol=err_rtol, atol=err_atol))
    min_err_index = np.where(np.equal(weighted_err,weighted_err[np.argmax(np.absolute(weighted_err-0.5))]))
    
    min_err_index=np.argmax(Margin==np.max(Margin[min_err_index]))
    min_err_Tau=Tau[min_err_index]
    min_err_Margin=Margin[min_err_index]
    false_neg=np.multiply(error_samples,(y_sorted==1)[np.newaxis,:]).sum(axis=1) #false neg
    false_pos=np.multiply(error_samples,(y_sorted!=1)[np.newaxis,:]).sum(axis=1) #false pos
    return_array = np.array([false_neg[min_err_index],false_pos[min_err_index],\
                     min_err_Margin,min_err_Tau,min_err_index,weighted_err[min_err_index],Toggle[min_err_index]])
    del Margin
    del Tau
    del error_samples
    del weighted_err
    del Toggle
    del feature_val_inds
    del feature_val
    del my_array
    del y_sorted
    return return_array


### Adaboost

In [ ]:
array_union=np.load('array_union.npy').astype('int32')
array_union_sel=np.empty((10,2)).astype('int32')
for k in range(1,10) :
        array_union_sel[k-1,:] = np.array([(k-1)*1000,k*1000]).astype('int32')
array_union_sel[-1,:]=np.array([k*1000,9358]).astype('int32')

In [ ]:

num_examples=1000
train_sel = np.reshape([1500+np.arange(num_examples/2),-np.arange(1,num_examples/2+1)[::-1]],-1).astype('int32')

y = np.concatenate((-1*np.ones(features_nonfaces.shape[0]),np.ones(features_faces.shape[0])))
X = np.vstack([features_nonfaces,features_faces])

X= X[train_sel,:]
y=y[train_sel]

#del features_nonfaces
#del features_faces

d=X.shape[1]
m=X.shape[0]

weights_init=(np.ones((m))/m)
feature_sel=np.empty((64,2)).astype('int32')

for k in range(1,64) :
        feature_sel[k-1,:] = np.array([(k-1)*1000,k*1000]).astype('int32')
feature_sel[-1,:]=np.array([k*1000,63960]).astype('int32')





num_iterations=5



In [ ]:
hypothesis_feature = []
hypothesis_Tau = []
hypothesis_Toggle = []
hypothesis_weights = []
training_false_neg=[]
training_false_pos=[]


weights = np.ones(m) / m
err_rtol=1e-02
err_atol=1e-08

false_neg_penalty=2
false_pos_penalty=1

for t in range(num_iterations):
    k=0
    print('Adaboost iteration:',t,', feature (x1000):',k)
    result_final=\
    np.apply_along_axis(lambda x: decision_stump(x,y,m,weights,false_neg_penalty,false_pos_penalty,\
                                                     err_atol=err_atol,err_rtol=err_rtol),axis=0,\
                        arr=X[:,array_union[range(array_union_sel[k,0],array_union_sel[k,1])]])
    k_upper=10
    for k in range(1,k_upper) :

        print('Adaboost iteration:',t,', feature (x1000):',k)
        result_1e4=\
        np.apply_along_axis(lambda x: decision_stump(x,y,m,weights,false_neg_penalty,false_pos_penalty,\
                                                     err_atol=err_atol,err_rtol=err_rtol),axis=0,\
                        arr=X[:,array_union[range(array_union_sel[k,0],array_union_sel[k,1])]])
        result_final = np.append(result_final,result_1e4,axis=1)
        del result_1e4
        
    err_rtol=err_rtol/1000
    err_atol=err_atol/1000
    
    min_err_feature = np.argmin(result_final[5,:])
    
    min_err_feature = np.where(np.isclose(result_final[5,:],\
                                                   np.min(result_final[5,:]),rtol=err_rtol, atol=err_atol))[0]
    
    #if len(min_err_feature) > 1:
    min_err_feature = min_err_feature[np.argmax(result_final[2,min_err_feature])]
    min_err_Tau=result_final[3,min_err_feature].astype('float')
    min_err_Toggle=result_final[6,min_err_feature].astype('bool')
    min_weighted_err_hypothesis=result_final[5,min_err_feature].astype('float')
    min_err_false_neg=result_final[0,min_err_feature].astype('float')
    min_err_false_pos=result_final[1,min_err_feature].astype('float')

    ### "!!! Use next line only for array_union !!!"
    ### "!!! **** !!!"
    min_err_feature = array_union[min_err_feature]
    
    error_samples=(np.bitwise_xor(np.greater_equal(\
                        np.reshape(X[:,min_err_feature],-1),min_err_Tau),min_err_Toggle)!=(y>0))  
    if t==0:
        error_samples2 = error_samples
    else:
        error_samples2 = np.bitwise_xor(np.multiply(2*reduce(np.bitwise_xor,(np.greater_equal(\
                                X[:,np.array(hypothesis_feature)],np.array(hypothesis_Tau)[np.newaxis,:])\
                ,np.array(hypothesis_Toggle)[np.newaxis,:]))-1,np.array(hypothesis_weights)[np.newaxis,:]).\
sum(axis=1)>=0,y>0)
        
    min_weighted_err=reduce(np.multiply,(error_samples,weights,np.where(y==1,false_neg_penalty,false_pos_penalty\
                                             ))).sum(axis=0)
    if min_weighted_err>0:
        alpha = (np.log(1 - min_weighted_err) - np.log(min_weighted_err)) / 2
    else:
        alpha=0
        t=num_iterations
    
    false_list=np.multiply(error_samples2,np.where(y==1,-10,10))
    
    print('accuracy:',1-np.sum(error_samples2>0)/1000,'FP:',np.sum(false_list==10),'FN:',np.sum(false_list==-10))
    weights = weights * np.exp(- alpha * (2*error_samples-1))
    weights = weights / weights.sum()
    
    hypothesis_feature.append(min_err_feature)
    hypothesis_Tau.append(min_err_Tau)
    hypothesis_Toggle.append(min_err_Toggle)                          
    hypothesis_weights.append(alpha)
    training_false_neg.append(min_err_false_neg)
    training_false_pos.append(min_err_false_pos)
y_pred=np.multiply(2*reduce(np.bitwise_xor,(np.greater_equal(\
                                X[:,np.array(hypothesis_feature)],np.array(hypothesis_Tau)[np.newaxis,:])\
                ,np.array(hypothesis_Toggle)[np.newaxis,:]))-1,np.array(hypothesis_weights)[np.newaxis,:]).\
sum(axis=1)>=0



In [ ]:
hypothesis_feature

In [ ]:
hypothesis_Tau

In [ ]:
#If toggle is 1 => hypothesis: y = 1 if feature_value<Threshold
hypothesis_Toggle

In [ ]:
hypothesis_weights

In [ ]:
training_false_neg

In [ ]:
training_false_pos